# Load in the Data Using Generator

Each sample in the dataset has a unique ID. In the following code, when defining the generator, we first get the list of ID's from the RGB folder, then use the list of ID's to load the corresponding information of each sample. Associated with each sample is the following information (size and directory name for the training set are also listed):
    1. RGB image (512 * 512 * 3) (train/images/rgb)
    2. NIR image (512 * 512 * 1) (train/images/rgb)
    3. Boundary (512 * 512 * 1) (train/boundaries)
    4. Mask (512 * 512 * 1) (train/masks)
The task of this project is to use the information above to predict for each pixel, which of the six categories it belons to:
    1. Cloud shadow (512 * 512 * 1) (train/labels/cloud_shadow)
    2. Double plant (512 * 512 * 1) (train/labels/double_plant)
    3. Planter skip (512 * 512 * 1) (train/labels/planter_skip)
    4. Standing water (512 * 512 * 1) (train/labels/standing_water)
    5. Waterway (512 * 512 * 1) (train/labels/waterway)
    6. Weed cluster (512 * 512 * 1) (train/labels/weed_cluster)
If a pixel does not belong to any category above, it is considered to be
    0. Background

In [ ]:
# import tensorflow as tf
import os
import numpy as np
from matplotlib import image as mpimg
from matplotlib import pyplot as plt


def id_check(path, ntail, ids):
    fignames = os.listdir(path)
    for fig_id in ids:
        if not fig_id + ntail in fignames:
            return False
    return True


def make_label(raw_labels):
    """
    This function creates a 7-channel label from the six labels (add a background channel).
    """
    stacked_labels = np.stack(raw_labels, axis=-1)
    last_label = np.sum(stacked_labels, axis=-1) + 1
    last_label[last_label > 1] = 0
    # Put the background as the last channel of the label
    full_labels =  np.concatenate([stacked_labels, last_label.reshape((512, 512, 1))], axis=-1)
    return full_labels / np.linalg.norm(full_labels, axis=-1).reshape((512, 512, 1))
    


def img_gen(dataset='train'):
    rgb_path = os.path.join(dataset, 'images', 'rgb')
    nir_path = os.path.join(dataset, 'images', 'nir')
    bdry_path = os.path.join(dataset, 'boundaries')
    mask_path = os.path.join(dataset, 'masks')
    label_names = ['cloud_shadow', 'double_plant', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']
    label_paths = [os.path.join(dataset, 'labels', label_name) for label_name in label_names]
    
    rgb_fig_names = os.listdir(rgb_path)
    fig_ids = [fname[:-4] for fname in rgb_fig_names]
    for fig_id in fig_ids:
        rgb_img = mpimg.imread(os.path.join(rgb_path, fig_id + '.jpg'))
        nir_img = mpimg.imread(os.path.join(nir_path, fig_id + '.jpg')).reshape((512, 512, 1))
        bdry_img = mpimg.imread(os.path.join(bdry_path, fig_id + '.png'))
        mask_img = mpimg.imread(os.path.join(mask_path, fig_id + '.png'))
        label_imgs = [mpimg.imread(os.path.join(label_path, fig_id + '.png')) for label_path in label_paths]
        input_img = np.concatenate([rgb_img, nir_img], axis=2) / 255. # Concatenate the RGB and NIR
        label = make_label(label_imgs)
        yield fig_id, input_img, bdry_img, mask_img, label

# Visualize a Training Sample

Next we see a visualization of the training sample. The first four images are RGB image, NIR image, boundary and mask. Multiplying the boundary with the mask gives you the "final mask", in which the pixels that you need to classify are marked white (with value 1), and the pixels you don't need to classify are marked black (with value 0). The next three images shows the RGB and the NIR image after you apply the boundary and the mask, and the "final mask".

Continuing to the next six images, these shows the labels corresponding to cloud shadow, double plant, planter skip, standing water, water way, and weed cluster. Pixels marked white (with value 1) in the image belongs to the category of the corresponding image. For instance, in the last label image (corresponding to the water way), the pixels that appears white belongs to the category of waterway.

The last image shows the background. A pixel belongs to the category of background if it does not belong to any of the six categories above.

You can change the "sample_idx" to see more examples.

# Handling the Labels

Pay attention to the "make_label" function in the first code block. After reading in the label of a sample, you will get six arrays each with size (512 * 512). However, it would be more handy to deal with an array of (512 * 512 * 7). This 7-channel array gives a one-hot encoding of each pixel's category.

In [ ]:
sample_idx = 6
cnt = 0
for fig_id, input_img, bdry_img, mask_img, label in img_gen():
    cnt += 1
    if cnt == sample_idx:
        print('Figure ID: %s' % fig_id)
        plt.figure(0)
        ax1 = plt.subplot(221)
        ax1.imshow(input_img[:, :, :3])
        ax2 = plt.subplot(222)
        ax2.imshow(input_img[:, :, 3], cmap='gray')
        ax3 = plt.subplot(223)
        ax3.imshow(bdry_img, cmap='gray', vmin=0, vmax=1)
        ax4 = plt.subplot(224)
        ax4.imshow(mask_img, cmap='gray', vmin=0, vmax=1)
        
        plt.figure(1)
        final_mask = np.multiply(bdry_img, mask_img) # The final mask is the intersection of the boundary and the mask.
        masked_img = np.multiply(input_img, final_mask.reshape((512, 512, 1)))
        ax1 = plt.subplot(131)
        ax1.imshow(masked_img[:, :, :3])
        ax2 = plt.subplot(132)
        ax2.imshow(masked_img[:, :, 3], cmap='gray')
        ax3 = plt.subplot(133)
        ax3.imshow(final_mask, cmap='gray')
        
        plt.figure(2)
        for i in range(6):
            ax = plt.subplot(231 + i)
            ax.imshow(label[:, :, i], cmap='gray', vmin=0, vmax=1)
        
        plt.figure(3)
        plt.imshow(label[:, :, 6], cmap='gray', vmin=0, vmax=1)
        break

# Handling the Boundary and the Mask

Many of you are confused about how to handle the boundary and the mask. If you read the instruction from git github repo (https://github.com/SHI-Labs/Agriculture-Vision), then you would see that the mask and the boundary pretty much has the same function: pixels that appear to be black in either mask and or boundary are the pixels that you don't need to classify for! Therefore, what we would like to do is to multiply the mask with the boundary to form the "final mask", as we did in the code above. This "final mask" contains all information from the mask and the boundary.

How to use the "final mask"? One option could be to multiply it with the RGB and NIR image. This "blacks out" all pixels that you don't need to consider in you classification. Although this is a convenient approach, it may gives you trouble when you start using more complicated methods like CNN, as the "blacked out" pixels are just zeros, and they still plays a roll in the whole network (affects the forward and backward process). Therefore, it is important to find a way to completely eliminate the influence of these pixels. This is a very important an interesting part of this project.

# Example: Dummy Classifier

In the following example, we take 10 samples from the training set, use a dummy classifier to train it, and compute loss with respect to the labels. The dummy classifier does nothing else but generate random classification, but this gives you a sense of what should be considered as input of the model, and what should be considered as output.

In our example we will just use the very basic method to handle the mask and the boundary: we multiply the image with the "final mask". Keep in mind that you are expected to explore more method of handling the mask and the boundary.

In [ ]:
class DummyClassifier:
    
    def __init__(self):
        # Randomly generate some parameters.
        self.param = np.random.normal(size=(4, 7))
        
    def softmax(self, logits):
        batch_size = logits.shape[0]
        exp_val = np.exp(logits)
        normalizer = np.sum(exp_val, axis=-1).reshape((batch_size, 512, 512, 1))
        return exp_val / normalizer
    
    def predict(self, X):
        result = self.softmax(X @ self.param)
        return result
    
def MSE(y_true, y_pred):
    """
    For simplicity, we use the MSE loss, but it is better to use cross entropy loss with softmax.
    """
    return np.sum(np.power(y_pred - y_true, 2)) / y_true.shape[0]


cnt = 0
X_list = []
y_list = []
fig_ids = []
for fig_id, input_img, bdry_img, mask_img, label in img_gen():
    cnt += 1
    fig_ids.append(fig_id)
    y_list.append(label) # 
    final_mask = np.multiply(bdry_img, mask_img).reshape(512, 512, 1) # Form the final mask
    masked_img = np.multiply(final_mask, input_img) # Multiply with the RGB and NIR image
    X_list.append(masked_img)
    if cnt == 10:
        break
X_batch = np.stack(X_list, axis=0)
y_batch = np.stack(y_list, axis=0)
# Check the shape of X and y.
print(X_batch.shape, y_batch.shape)

# Initialize a classifier
classifier = DummyClassifier()
# Predict on X_batch
y_pred = classifier.predict(X_batch)
print(y_pred.shape) # Expected to be of the same shape as y_batch
# Compute the error
error = MSE(y_batch, y_pred)
print(error) # Expected to be very large since we are doing random classification.

# Save the Predictions

The next step is to save the 10 predictions under the name of their ID. Before saving the result, we first need to convert the 7-channel prediction (one-hot) into the 1-channel representation. Note that when generate the background label we appended as the 7th channel. Therefore, when converting to 1-channel representation, it would be handy to first bring that channel to the first channel, so that the channel index of each category matches the coresponding value indicator (0-6) of each category. The 10 predictions are saved under a directory named "pred_example".

In [ ]:
from PIL import Image

if not os.path.isdir('pred_example'):
    os.mkdir('pred_example')

for i in range(10):
    pred_result = y_pred[i] # take the ith predictio
    # Since we appended the background to the last channel, we need to bring it to the front when saving predictions
    processed_result = np.concatenate([pred_result[:, :, -1].reshape((512, 512, 1)), pred_result[:, :, :-1]], axis=-1)
    print(processed_result.shape) # Should be (512, 512, 7)
    # Convert the 7-channel result to 1-channel result and cast to uint8
    final_pred = np.argmax(processed_result, axis=-1).astype(np.uint8)
    print(final_pred.shape) # Should be (512, 512)
    # Save the prediction
    filename = os.path.join('pred_example', fig_ids[i] + '.png')
    Image.fromarray(final_pred).save(filename)

# Zip the Result

Finally, zip the files in the "pred_example" folder using:\
   ```zip submission.zip *.png```\
The result can be submitted to the server for evaluation (here we are predicting only 10 samples for the training set. For the server to process your submission, you need to predict all samples in the test set). 